In [68]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
# target value proportion e.g. accuracy baseline
train['Exited'].value_counts() / train.shape[0]

0    0.795546
1    0.204454
Name: Exited, dtype: float64

In [60]:
from sklearn.preprocessing import MinMaxScaler 

train = pd.read_csv("data/Task_2/account_histroy_data.csv", index_col=1).drop("RowNumber", axis=1)
test = pd.read_csv("data/Task_2/existing_account.csv", index_col=0)

# drop unused columns
drop_cols = ["Surname"]
train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)

# binarize categorical columns
cat_cols = ["Geography", "Gender"]
for column in cat_cols:
    train = pd.merge(
        train.drop(column, axis=1),
        pd.get_dummies(train[column]).add_prefix(column + "_"),
        left_index=True, right_index=True
    )
    test = pd.merge(
        test.drop(column, axis=1),
        pd.get_dummies(test[column]).add_prefix(column + "_"),
        left_index=True, right_index=True
    )
    
    
# normalize numerical columns
num_col = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = MinMaxScaler()
scaler.fit(pd.concat([train[num_col], test[num_col]]))
train.loc[:, num_col] = scaler.transform(train.loc[:, num_col])
test.loc[:, num_col] = scaler.transform(test.loc[:, num_col])

train.head()

/Users/westi/Repos/infotest/env/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
CustomerId,,,,,,,,,,,,,,
15634602,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,1,0,0,1,0
15647311,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1,1,0
15619304,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,1,0,0,1,0
15701354,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0,1,0
15737888,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1,1,0


In [61]:
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, accuracy_score, recall_score, cohen_kappa_score

metric_funcs = {
    "log_loss": log_loss,
    "recall_score": recall_score,
    "accuracy_score": accuracy_score,
    "cohen_kappa_score": cohen_kappa_score
}


def cross_validate(Model, params, data, t_col = "Exited"):
    log = []
    for i, (tr_i, t_i) in enumerate(KFold(n_splits=5).split(data)):
        X_tr, y_tr = data.drop(t_col, axis=1).iloc[tr_i], data[t_col].iloc[tr_i]
        X_t, y_t = data.drop(t_col, axis=1).iloc[t_i], data[t_col].iloc[t_i]
        model = Model(**params)
        model.fit(X_tr, y_tr)
        pred = model.predict(X_t)
        log.append({
            **{n: f(y_t, pred) for n, f in metric_funcs.items()},
            **params
        })
        
    log_df = pd.DataFrame(log).mean().to_frame().transpose()
    for k, v in params.items():
        log_df[k] = str(v)
        
    return log_df

In [62]:
from multiprocessing import cpu_count

CORES = cpu_count()

# sample from generated parameter space for cv
def sample(space):
    params = {}
    for k in space.keys():
        params[k] = random.choice(space[k])

    return params

# iterate over param spaces and aggregate results
def cv_for_params(Model, param_space, param_statics):
    return pd.concat([
        cross_validate(
            Model,
            {**sample(param_space), **param_statics},
            train
        ) for i in range(N)
    ]).reset_index(drop=True)

In [53]:
from sklearn.neighbors import KNeighborsClassifier

N = 5

param_space = {
    'n_neighbors': list(range(1, 25)),
}
param_statics = {'n_jobs': CORES}

knn_results = cv_for_params(
    KNeighborsClassifier,
    param_space,
    param_statics,
)
knn_results.sort_values("accuracy_score", ascending=False).head(5)

,accuracy_score,cohen_kappa_score,log_loss,n_jobs,n_neighbors,recall_score
3,0.817302,0.273532,6.310182,4,9,0.248488
4,0.817199,0.265303,6.313753,4,11,0.237310
1,0.809466,0.171529,6.580807,4,20,0.139759
2,0.809466,0.171529,6.580807,4,20,0.139759
0,0.808744,0.177172,6.605745,4,23,0.148321


In [55]:
from sklearn.linear_model import LogisticRegression

N = 3

param_space = {
    'class_weight': [None, 'balanced'],
    'solver': ['lbfgs', 'saga']
}
param_statics = {'n_jobs': CORES}

log_results = cv_for_params(
    LogisticRegression,
    param_space,
    param_statics,
)
log_results.sort_values("accuracy_score", ascending=False).head(5)

/Users/westi/Repos/infotest/env/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,accuracy_score,class_weight,cohen_kappa_score,log_loss,n_jobs,recall_score,solver
0,0.810806,None,0.228845,6.534566,4,0.206753,saga
1,0.810806,None,0.228845,6.534566,4,0.206753,saga
2,0.709865,balanced,0.313935,10.021081,4,0.694467,saga


In [63]:
from xgboost import XGBClassifier

N = 50

param_space = {
    'learning_rate': np.geomspace(1e-2, 1),
    'max_depth': list(range(1, 10)),
    'gamma': np.geomspace(1e-2, 1),
    'min_child_weight': list(range(1, 10)),
    'num_estimators': list(range(30, 300)),
    'reg_alpha': np.linspace(0.2, 1),
    'reg_lambda': np.linspace(0.2, 2),
    'scale_pos_weight': np.linspace(0.3, 2)
}
param_statics = {'n_jobs': CORES}

xgb_results = cv_for_params(
    XGBClassifier,
    param_space,
    param_statics,
)
xgb_results.sort_values("accuracy_score", ascending=False).head(5)

,accuracy_score,cohen_kappa_score,gamma,learning_rate,log_loss,max_depth,min_child_weight,n_jobs,num_estimators,recall_score,reg_alpha,reg_lambda,scale_pos_weight
11,0.863904,0.499208,0.021209508879201904,0.04941713361323833,4.700607,7,2,4,111,0.446775,0.7061224489795919,0.5306122448979592,0.8204081632653062
3,0.863697,0.506529,0.013257113655901088,0.42919342601287763,4.707759,2,8,4,165,0.464353,0.2653061224489796,1.0081632653061225,0.8551020408163266
18,0.863078,0.511068,0.32374575428176433,0.22229964825261944,4.729126,3,7,4,114,0.478580,0.4448979591836735,1.6326530612244898,0.9938775510204081
5,0.862254,0.502612,0.08685113737513525,0.22229964825261944,4.757613,5,2,4,63,0.463917,0.2326530612244898,1.1918367346938776,0.7857142857142857
38,0.861532,0.496564,0.7543120063354615,0.017575106248547922,4.782547,8,2,4,144,0.454338,0.7224489795918367,0.7510204081632654,0.9244897959183673


In [80]:
final_model = XGBClassifier(**{
    'gamma': 0.3237,
    'learning_rate': 0.2223,
    'max_depth': 3,
    'n_jobs': 4,
    'num_estimators': 114,
    'min_child_weight': 7,
    'reg_alpha': 0.445,
    'reg_lambda': 1.63265,
    'scale_pos_weight': 0.994
})
final_model.fit(train.drop("Exited", axis=1), train["Exited"])
predictions = pd.Series(final_model.predict(test), index=test.index)
pd.concat([
    pd.read_csv("data/Task_2/existing_account.csv", index_col=0),
    predictions.rename("Exited").to_frame()
], axis=1).to_csv('data/Task_2/predictions.csv')
predictions.value_counts() / predictions.shape[0]

0    0.916944
1    0.083056
dtype: float64